In [ ]:
import os
import time
import boto3
import logging

In [ ]:
logging.getLogger().setLevel(logging.INFO)
logger = logging.getLogger('mxnet_data')
fh = logging.FileHandler('data-publishing.log')
fh.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
logger.addHandler(fh)

In [ ]:
def get_queue_url(sqs, name):
    response = sqs.list_queues()
    for url in response['QueueUrls']:
        if url.find('/' + name + '.fifo') != -1 or url.find('/' + name)!= -1:
            return url

In [ ]:
def find_last(text, token):
    position = []
    for n in range(len(text)):
        if text.find(token, n, n+1) != -1:
            position.append(n)
    return position[len(position)-1]

In [ ]:
def publish_to_queue(sqs, queue, filename):
    data_file = open(filename, 'r')
    data_list = data_file.readlines()
    data_file.close()    
    message = ''
    for record in data_list:
        message = message + record
    response = sqs.send_message(
        QueueUrl=queue,
        MessageBody=message,
        MessageDeduplicationId=filename[find_last(filename, '_')+1:find_last(filename, '.')],
        MessageGroupId=filename[find_last(filename, '/')+1:find_last(filename, '_')]
    )
    return response

In [ ]:
sqs = boto3.client('sqs', 'us-east-1')

In [ ]:
queue = get_queue_url(sqs, 'training-data-queue')
logger.info('Publishing Queue URL - ' + queue)

In [ ]:
lines_per_file = 120
smallfile = None
progress = 0
if os.path.exists('check.dat'):
    check = open('check.dat','r')
    progress = int(check.readlines()[0])
    check.close()
with open('./dataset/mnist_train.csv') as bigfile:
    for lineno, line in enumerate(bigfile):
        if int(lineno / lines_per_file) < progress:
            continue
        if lineno % lines_per_file == 0:
            if smallfile:
                smallfile.close()
                response = publish_to_queue(sqs,queue,small_filename)
                logger.info('Data Packet - ' + small_filename[find_last(small_filename, '/')+1:find_last(small_filename, '.')]+ ' published at ' + response['ResponseMetadata']['HTTPHeaders']['date'])
                os.remove(small_filename)
                check = open('check.dat','w') 
                check.write(str(int(small_filename[find_last(small_filename, '_')+1:find_last(small_filename, '.')])))
                check.close()
                time.sleep(30)                
            small_filename = './dataset/incremental/mnist_train_{}.csv'.format(str(int(lineno / lines_per_file) + 1).zfill(3))
            smallfile = open(small_filename, "w")
        smallfile.write(line)
    if smallfile:
        smallfile.close()
if os.path.exists('check.dat'):
    os.remove('check.dat')